In [19]:
import os
import time
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from shapely.geometry import Polygon
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

In [20]:
class OccupancyDetectionTest:
    '''Class for testing the accuracy of the Parking Occupancy Detection (detect.py)'''
    
    @staticmethod
    def calculate_iou(parking_spaces, car_boxes):

        new_car_boxes = []
        for box in car_boxes:
            x1, y2, x2, y1 = box
            new_car_boxes.append([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])

        iou_matrix = np.zeros((len(parking_spaces), len(new_car_boxes)))   # creates matrix, where columns are the parking slots
                                                                         # and the rows are the detected cars
        for i in range(len(parking_spaces)):                             # for each parking slot
            for j in range(len(new_car_boxes)):                          # for each detected car
                polygon1 = Polygon(parking_spaces[i])                     # create polygon for parking space
                polygon2 = Polygon(new_car_boxes[j])                     # create polygon for bounding box of a car
                polygon_intersection = polygon1.intersection(polygon2).area
                polygon_union = polygon1.union(polygon2).area
                iou_matrix[i][j] = polygon_intersection/polygon_union
        return iou_matrix

    @staticmethod
    def frame_detect(model, frame, parking_spaces, threshold=0.1):
        img_detected_spaces = []

        cars = model.predict(frame)[0].boxes.xyxy
        iou_matrix = OccupancyDetectionTest.calculate_iou(parking_spaces, cars)

        for parking_space, cars_iou in zip(parking_spaces, iou_matrix):
            max_iou = np.max(cars_iou)
            if max_iou < threshold:  
                img_detected_spaces.append(0)
            else:
                img_detected_spaces.append(1)
        return img_detected_spaces
    
    @staticmethod
    def get_detected(parking_spaces_path, images_path, threshold):
        with open(parking_spaces_path, 'rb') as f:
            parking_spaces = np.load(f)

        images = os.listdir(images_path)
        images.sort()
        detected_spaces = []
        for image in images:
            img = cv2.imread(images_path+'/'+image)
            detected = OccupancyDetectionTest.frame_detect(model, img, parking_spaces, threshold)
            for class_ in detected:
                detected_spaces.append(int(class_))
        return detected_spaces


In [21]:
os.chdir('..')

In [22]:
# Detects and stores all the class ids in a list

parking_spaces_path = 'occupancy_detection/parking_spaces.npy'
images_path = 'datasets/occupancy_test/test/images'
model = YOLO('object_detection/models/detect/singleclass/weights/yolov8_cars.pt')

detected_spaces_01 = OccupancyDetectionTest.get_detected(parking_spaces_path, images_path, 0.01)
detected_spaces_05 = OccupancyDetectionTest.get_detected(parking_spaces_path, images_path, 0.05)
detected_spaces_1 = OccupancyDetectionTest.get_detected(parking_spaces_path, images_path, 0.1)
detected_spaces_15 = OccupancyDetectionTest.get_detected(parking_spaces_path, images_path, 0.15)
detected_spaces_2 = OccupancyDetectionTest.get_detected(parking_spaces_path, images_path, 0.2)


0: 384x640 7 cars, 172.1ms
Speed: 2.0ms preprocess, 172.1ms inference, 5.8ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 7 cars, 155.8ms
Speed: 1.4ms preprocess, 155.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 6 cars, 155.2ms
Speed: 1.1ms preprocess, 155.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 165.3ms
Speed: 0.9ms preprocess, 165.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 153.7ms
Speed: 1.1ms preprocess, 153.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 158.8ms
Speed: 1.3ms preprocess, 158.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 6 cars, 158.5ms
Speed: 1.5ms preprocess, 158.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 7 cars, 159.1ms
Speed: 1.1ms preprocess, 159.1ms inference, 0.9ms postprocess per image at shape (1, 3, 64

Speed: 0.9ms preprocess, 153.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 147.8ms
Speed: 1.1ms preprocess, 147.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 146.2ms
Speed: 0.8ms preprocess, 146.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 144.2ms
Speed: 0.7ms preprocess, 144.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 146.7ms
Speed: 0.7ms preprocess, 146.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 147.9ms
Speed: 0.8ms preprocess, 147.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 146.8ms
Speed: 0.8ms preprocess, 146.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 146.8ms
Speed: 0.7ms preprocess, 146.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 


0: 384x640 2 cars, 152.2ms
Speed: 0.6ms preprocess, 152.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 149.2ms
Speed: 0.8ms preprocess, 149.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 147.5ms
Speed: 0.7ms preprocess, 147.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 144.9ms
Speed: 1.1ms preprocess, 144.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 146.4ms
Speed: 0.7ms preprocess, 146.4ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 144.5ms
Speed: 0.9ms preprocess, 144.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 145.5ms
Speed: 0.7ms preprocess, 145.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 144.9ms
Speed: 1.0ms preprocess, 144.9ms inference, 1.0ms postprocess per image at shape (1, 3, 64


0: 384x640 9 cars, 159.0ms
Speed: 1.3ms preprocess, 159.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 11 cars, 154.5ms
Speed: 0.8ms preprocess, 154.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 9 cars, 148.9ms
Speed: 0.8ms preprocess, 148.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 9 cars, 158.3ms
Speed: 1.4ms preprocess, 158.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 9 cars, 162.3ms
Speed: 0.8ms preprocess, 162.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 10 cars, 161.3ms
Speed: 1.0ms preprocess, 161.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 10 cars, 151.1ms
Speed: 1.0ms preprocess, 151.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 11 cars, 148.5ms
Speed: 0.8ms preprocess, 148.5ms inference, 1.0ms postprocess per image at shape (1, 3


0: 384x640 4 cars, 147.7ms
Speed: 0.7ms preprocess, 147.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 145.5ms
Speed: 0.8ms preprocess, 145.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 147.8ms
Speed: 0.8ms preprocess, 147.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 147.1ms
Speed: 1.7ms preprocess, 147.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 145.7ms
Speed: 0.7ms preprocess, 145.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 146.5ms
Speed: 1.0ms preprocess, 146.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 146.6ms
Speed: 0.7ms preprocess, 146.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 147.4ms
Speed: 1.1ms preprocess, 147.4ms inference, 1.0ms postprocess per image at shape (1, 3, 64

...


0: 384x640 2 cars, 148.4ms
Speed: 0.7ms preprocess, 148.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 146.0ms
Speed: 0.7ms preprocess, 146.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 145.8ms
Speed: 0.7ms preprocess, 145.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 145.0ms
Speed: 0.7ms preprocess, 145.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 145.8ms
Speed: 0.7ms preprocess, 145.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 150.7ms
Speed: 0.8ms preprocess, 150.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 146.4ms
Speed: 0.7ms preprocess, 146.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 cars, 146.2ms
Speed: 0.8ms preprocess, 146.2ms inference, 1.0ms postprocess per image at shape (1, 3, 64


0: 384x640 9 cars, 147.7ms
Speed: 1.3ms preprocess, 147.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 9 cars, 144.4ms
Speed: 0.8ms preprocess, 144.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 9 cars, 145.5ms
Speed: 0.8ms preprocess, 145.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 10 cars, 145.7ms
Speed: 0.8ms preprocess, 145.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 10 cars, 146.6ms
Speed: 0.7ms preprocess, 146.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 11 cars, 146.8ms
Speed: 0.8ms preprocess, 146.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 11 cars, 146.1ms
Speed: 1.0ms preprocess, 146.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 11 cars, 146.4ms
Speed: 1.0ms preprocess, 146.4ms inference, 1.0ms postprocess per image at shape (1, 


0: 384x640 3 cars, 156.8ms
Speed: 0.9ms preprocess, 156.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 154.3ms
Speed: 1.1ms preprocess, 154.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 154.8ms
Speed: 0.7ms preprocess, 154.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 150.7ms
Speed: 0.8ms preprocess, 150.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 152.1ms
Speed: 0.7ms preprocess, 152.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 154.6ms
Speed: 0.7ms preprocess, 154.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 152.4ms
Speed: 0.8ms preprocess, 152.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 153.5ms
Speed: 0.7ms preprocess, 153.5ms inference, 1.0ms postprocess per image at shape (1, 3, 64

Speed: 0.7ms preprocess, 157.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 157.8ms
Speed: 0.7ms preprocess, 157.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 154.5ms
Speed: 0.8ms preprocess, 154.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 158.3ms
Speed: 0.8ms preprocess, 158.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 6 cars, 165.8ms
Speed: 0.8ms preprocess, 165.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 163.5ms
Speed: 1.2ms preprocess, 163.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 202.8ms
Speed: 0.9ms preprocess, 202.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 169.0ms
Speed: 1.2ms preprocess, 169.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 


0: 384x640 6 cars, 156.6ms
Speed: 0.9ms preprocess, 156.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 7 cars, 156.6ms
Speed: 0.8ms preprocess, 156.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 6 cars, 151.3ms
Speed: 0.9ms preprocess, 151.3ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 6 cars, 145.7ms
Speed: 0.7ms preprocess, 145.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 145.5ms
Speed: 0.7ms preprocess, 145.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 147.6ms
Speed: 0.8ms preprocess, 147.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 144.5ms
Speed: 1.0ms preprocess, 144.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 147.4ms
Speed: 0.8ms preprocess, 147.4ms inference, 1.0ms postprocess per image at shape (1, 3, 64

In [23]:
# Stores all the ground truth class ids in a list

gt_spaces = []

labels = os.listdir('datasets/occupancy_test/test/labels')
labels.sort()
for txt_file in labels:
    with open('datasets/occupancy_test/test/labels/'+txt_file, 'r') as file:
        data = file.readlines()
        for space in data:
            gt_spaces.append(1-int(space[0]))

In [37]:
score_01 = accuracy_score(gt_spaces, detected_spaces_01)
prec_01, rec_01, f1_01, sp_01 = precision_recall_fscore_support(gt_spaces, detected_spaces_01, average='macro')

score_05 = accuracy_score(gt_spaces, detected_spaces_05)
prec_05, rec_05, f1_05, sp_05 = precision_recall_fscore_support(gt_spaces, detected_spaces_05, average='macro')

prec_1, rec_1, f1_1, sp_1 = precision_recall_fscore_support(gt_spaces, detected_spaces_1, average='macro')
score_1 = accuracy_score(gt_spaces, detected_spaces_1)

prec_15, rec_15, f1_15, sp_15 = precision_recall_fscore_support(gt_spaces, detected_spaces_15, average='macro')
score_15 = accuracy_score(gt_spaces, detected_spaces_15)

prec_2, rec_2, f1_2, sp_2 = precision_recall_fscore_support(gt_spaces, detected_spaces_2, average='macro')
score_2 = accuracy_score(gt_spaces, detected_spaces_2)


scores = [score_01, score_05, score_1, score_15, score_2]
precisions = [prec_01, prec_05, prec_1, prec_15, prec_2]
recalls = [rec_01, rec_05, rec_1, rec_15, rec_2]
f1s = [f1_01, f1_05, f1_1, f1_15, f1_2]

d = {}
d['Threshold  '] = ['0.01  ','0.05  ', '0.10  ', '0.15  ', '0.20  ']
d[' Accuracy'] = np.around(scores, 3)
d['Precision'] = np.around(precisions, 3)
d['Recall'] = np.around(recalls, 3)
d['F1-score'] = np.around(f1s, 3)

df = pd.DataFrame(data=d)
df

,Threshold,Accuracy,Precision,Recall,F1-score
0,0.01,0.999,0.999,0.999,0.999
1,0.05,1.000,1.000,1.000,1.000
2,0.10,1.000,1.000,1.000,1.000
3,0.15,0.749,0.831,0.753,0.734
4,0.20,0.677,0.802,0.683,0.645


In [32]:
d1 = {}
d1['Class  '] = ['all  ','buss  ', 'car  ', 'truck  ']
d1['Images'] = [217, 217, 217, 217]
d1['Instances'] = [1116, 4, 1080, 32]
d1['mAP50'] = [0.806, 0.649, 0.959, 0.809]
d1['mAP50-95'] = [0.592, 0.49, 0.766, 0.52]
pd.DataFrame(data=d1)

,Class,Images,Instances,mAP50,mAP50-95
0,all,217,1116,0.806,0.592
1,buss,217,4,0.649,0.490
2,car,217,1080,0.959,0.766
3,truck,217,32,0.809,0.520


In [18]:
d = {}
d['Class '] = ['all','car']
d['Images'] = [217, 217]
d['Instances'] = [1116, 1116]
d['mAP50'] = [0.963, 0.963]
d['mAP50-95'] = [0.792, 0.792]

df = pd.DataFrame(data=d)
df

,Class,Images,Instances,mAP50,mAP50-95
0,all,217,1116,0.963,0.792
1,car,217,1116,0.963,0.792
